In [5]:
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
from torchvision import models, transforms
from torchvision.models import resnet18

In [6]:
def resize_image(input_path, size=(224, 224)):
    # Abrir la imagen
    image = Image.open(input_path).convert("L")  # Convertir a escala de grises
    
    # Obtener tamaño original
    original_width, original_height = image.size
    aspect_ratio = original_width / original_height
    
    # Calcular nuevo tamaño
    if aspect_ratio > 1:
        # Imagen más ancha que alta
        new_width = size[0]
        new_height = int(new_width / aspect_ratio)
    else:
        # Imagen más alta que ancha
        new_height = size[1]
        new_width = int(new_height * aspect_ratio)
    
    # Redimensionar la imagen manteniendo la relación de aspecto
    # si la imagen es mas chica que el tamaño deseado, uso interpolación LANCZOS, sino LINEAL
    if original_width < size[0] or original_height < size[1]:
        resized_image = image.resize((new_width, new_height), Image.LANCZOS)
    else:
        resized_image = image.resize((new_width, new_height), Image.LINEAR)
    
    # Crear un nuevo fondo de 224x224 y pegar la imagen redimensionada en el centro
    new_image = Image.new("L", size)
    new_image.paste(resized_image, ((size[0] - new_width) // 2, (size[1] - new_height) // 2))
    
    return new_image

In [4]:
# show original image
image = Image.open("CelebA/celebA/img_align_celeba/img_align_celeba/125478.jpg")
image.show()

# image size
print("original image size:", image.size)

# resize image
resized_image = resize_image("CelebA/celebA/img_align_celeba/img_align_celeba/125478.jpg")
resized_image.show()

# resized image size
print("resized image size:", resized_image.size)

original image size: (178, 218)
resized image size: (224, 224)


In [5]:
# range of pixel values of the original image
print("original image pixel range:", min(image.getdata()), max(image.getdata()))

# range of pixel values of the resized image
print("resized image pixel range:", min(resized_image.getdata()), max(resized_image.getdata()))

original image pixel range: (4, 0, 0) (255, 226, 234)
resized image pixel range: 0 236


## Resnet18

In [14]:
class SuperResolutionResNet(nn.Module):
    def __init__(self):
        super(SuperResolutionResNet, self).__init__()
        # Cargar el modelo preentrenado resnet18
        self.resnet = models.resnet18(pretrained=True)
        
        # Modificar la primera capa para aceptar una sola canal (imágenes en escala de grises)
        self.resnet.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        
        # Eliminar la última capa (clasificación)
        self.resnet = nn.Sequential(*list(self.resnet.children())[:-2])
        
        # Añadir capas para aumentar la resolución
        self.upconv1 = nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1)
        self.upconv2 = nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1)
        self.upconv3 = nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1)
        self.upconv4 = nn.ConvTranspose2d(64, 1, kernel_size=4, stride=2, padding=1)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.resnet(x)
        x = self.relu(self.upconv1(x))
        x = self.relu(self.upconv2(x))
        x = self.relu(self.upconv3(x))
        x = self.upconv4(x)
        return x

def resize_image(input_path, size=(224, 224)):
    # Abrir la imagen
    image = Image.open(input_path).convert("L")  # Convertir a escala de grises
    
    # Obtener tamaño original
    original_width, original_height = image.size
    aspect_ratio = original_width / original_height
    
    # Calcular nuevo tamaño
    if aspect_ratio > 1:
        # Imagen más ancha que alta
        new_width = size[0]
        new_height = int(new_width / aspect_ratio)
    else:
        # Imagen más alta que ancha
        new_height = size[1]
        new_width = int(new_height * aspect_ratio)
    
    # Redimensionar la imagen manteniendo la relación de aspecto
    # si la imagen es mas chica que el tamaño deseado, uso interpolación LANCZOS, sino LINEAL
    if original_width < size[0] or original_height < size[1]:
        resized_image = image.resize((new_width, new_height), Image.LANCZOS)
    else:
        resized_image = image.resize((new_width, new_height), Image.LINEAR)
    
    # Crear un nuevo fondo de 224x224 y pegar la imagen redimensionada en el centro
    new_image = Image.new("L", size)
    new_image.paste(resized_image, ((size[0] - new_width) // 2, (size[1] - new_height) // 2))
    
    return new_image

def super_resolve_image(image, model, device):
    # Convertir imagen a tensor y añadir un canal
    image = transforms.ToTensor()(image).unsqueeze(0).to(device)
    
    # Aumentar la resolución usando el modelo de super-resolución
    with torch.no_grad():
        sr_image = model(image)
    
    # Convertir de nuevo a PIL
    sr_image = sr_image.squeeze(0).cpu().numpy()
    sr_image = (sr_image - sr_image.min()) / (sr_image.max() - sr_image.min())  # Normalizar
    sr_image = (sr_image * 255).astype(np.uint8)  # Escalar a [0, 255]
    sr_image = Image.fromarray(sr_image[0])  # Convertir el primer canal a PIL
    return sr_image

def process_image(input_path, output_path):
    # Configuración del dispositivo
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Paso 1: Redimensionar la imagen manteniendo la relación de aspecto
    resized_image = resize_image(input_path, size=(224, 224))
    
    # Crear el modelo de super-resolución y cargarlo en el dispositivo
    model = SuperResolutionResNet().to(device)
    
    # Paso 2: Mejorar la resolución de la imagen redimensionada
    final_image = super_resolve_image(resized_image, model, device)
    
    # Guardar la imagen final
    final_image.save(output_path)

In [15]:
process_image("CelebA/celebA/img_align_celeba/img_align_celeba/125479.jpg", "resnet18_output_image.jpg")

## CNN

In [3]:
class SimpleSRCNN(nn.Module):
    def __init__(self):
        super(SimpleSRCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=9, padding=4)
        self.conv2 = nn.Conv2d(64, 32, kernel_size=1, padding=0)
        self.conv3 = nn.Conv2d(32, 1, kernel_size=5, padding=2)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.conv3(x)
        return x

def super_resolve_image(image, model):
    # Convertir imagen a tensor y añadir un canal
    image = transforms.ToTensor()(image).unsqueeze(0)
    
    # Aumentar la resolución usando el modelo de super-resolución
    with torch.no_grad():
        sr_image = model(image)
    
    # Convertir de nuevo a PIL
    sr_image = transforms.ToPILImage()(sr_image.squeeze(0))
    
    return sr_image

def process_image(input_path, output_path):
    # Paso 1: Redimensionar la imagen manteniendo la relación de aspecto
    resized_image = resize_image(input_path, size=(224, 224))
    
    # Crear el modelo de super-resolución
    model = SimpleSRCNN()
    
    # Cargar pesos preentrenados si tienes alguno (esto es solo un placeholder)
    # model.load_state_dict(torch.load('path_to_pretrained_weights.pth'))
    
    # Paso 2: Mejorar la resolución de la imagen redimensionada
    final_image = super_resolve_image(resized_image, model)
    
    # Guardar la imagen final
    final_image.save(output_path)

In [6]:
process_image("CelebA/celebA/img_align_celeba/img_align_celeba/125479.jpg", "output_image.jpg")

[W NNPACK.cpp:53] Could not initialize NNPACK! Reason: Unsupported hardware.


In [7]:
import torch
from PIL import Image
from torchvision.transforms import ToPILImage, ToTensor, Normalize
from basicsr.archs.rrdbnet_arch import RRDBNet
from basicsr.utils import imwrite

def resize_image(input_path, size=(224, 224)):
    # Abrir la imagen
    image = Image.open(input_path).convert("L")  # Convertir a escala de grises
    
    # Obtener tamaño original
    original_width, original_height = image.size
    aspect_ratio = original_width / original_height
    
    # Calcular nuevo tamaño
    if aspect_ratio > 1:
        # Imagen más ancha que alta
        new_width = size[0]
        new_height = int(new_width / aspect_ratio)
    else:
        # Imagen más alta que ancha
        new_height = size[1]
        new_width = int(new_height * aspect_ratio)
    
    # Redimensionar la imagen manteniendo la relación de aspecto
    # si la imagen es mas chica que el tamaño deseado, uso interpolación LANCZOS, sino LINEAL
    if original_width < size[0] or original_height < size[1]:
        resized_image = image.resize((new_width, new_height), Image.LANCZOS)
    else:
        resized_image = image.resize((new_width, new_height), Image.LINEAR)
    
    # Crear un nuevo fondo de 224x224 y pegar la imagen redimensionada en el centro
    new_image = Image.new("L", size)
    new_image.paste(resized_image, ((size[0] - new_width) // 2, (size[1] - new_height) // 2))
    
    return new_image

def super_resolve_image(image, model, device):
    # Convertir imagen a tensor y normalizar
    image = ToTensor()(image).unsqueeze(0).to(device)
    image = image.repeat(1, 3, 1, 1)  # Convertir a 3 canales replicando el canal de grises
    
    # Aumentar la resolución usando el modelo de super-resolución
    with torch.no_grad():
        sr_image = model(image)
    
    # Convertir de nuevo a PIL y a escala de grises
    sr_image = sr_image.squeeze(0).cpu()
    sr_image = ToPILImage()(sr_image[0])  # Convertir solo el primer canal a escala de grises
    
    return sr_image

def process_image(input_path, output_path, model_path):
    # Configuración del dispositivo
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Paso 1: Redimensionar la imagen manteniendo la relación de aspecto
    resized_image = resize_image(input_path, size=(224, 224))
    
    # Cargar el modelo de super-resolución ESRGAN
    model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=4)
    model.load_state_dict(torch.load(model_path), strict=True)
    model.eval()
    model = model.to(device)
    
    # Paso 2: Mejorar la resolución de la imagen redimensionada
    final_image = super_resolve_image(resized_image, model, device)
    
    # Guardar la imagen final
    final_image.save(output_path)

# Ejemplo de uso
process_image("CelebA/celebA/img_align_celeba/img_align_celeba/125479.jpg", "ESRGAN_output_image.jpg", "RealESRGAN_x4plus.pth")


RuntimeError: Error(s) in loading state_dict for RRDBNet:
	Missing key(s) in state_dict: "conv_first.weight", "conv_first.bias", "body.0.rdb1.conv1.weight", "body.0.rdb1.conv1.bias", "body.0.rdb1.conv2.weight", "body.0.rdb1.conv2.bias", "body.0.rdb1.conv3.weight", "body.0.rdb1.conv3.bias", "body.0.rdb1.conv4.weight", "body.0.rdb1.conv4.bias", "body.0.rdb1.conv5.weight", "body.0.rdb1.conv5.bias", "body.0.rdb2.conv1.weight", "body.0.rdb2.conv1.bias", "body.0.rdb2.conv2.weight", "body.0.rdb2.conv2.bias", "body.0.rdb2.conv3.weight", "body.0.rdb2.conv3.bias", "body.0.rdb2.conv4.weight", "body.0.rdb2.conv4.bias", "body.0.rdb2.conv5.weight", "body.0.rdb2.conv5.bias", "body.0.rdb3.conv1.weight", "body.0.rdb3.conv1.bias", "body.0.rdb3.conv2.weight", "body.0.rdb3.conv2.bias", "body.0.rdb3.conv3.weight", "body.0.rdb3.conv3.bias", "body.0.rdb3.conv4.weight", "body.0.rdb3.conv4.bias", "body.0.rdb3.conv5.weight", "body.0.rdb3.conv5.bias", "body.1.rdb1.conv1.weight", "body.1.rdb1.conv1.bias", "body.1